In [ ]:
dfrom google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np
import cv2

from tqdm import tqdm
import pickle

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

# Read data

In [ ]:
path = '/content/drive/MyDrive/COURSES/DS102-final-project/balls/'
ball_type = [_ for _ in os.listdir(path)]

ball_type

['1',
 '0',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15']

In [ ]:
df = {'file_path': [], 'label': []}
df = pd.DataFrame(df) 
for i in range(len(ball_type)):
    for file in os.listdir(path+ball_type[i]):
        new_row = {
                    'file_path': path + ball_type[i] + '/' + file,
                    'label': ball_type[i]
                  }
        df = df.append(new_row, ignore_index=True)
df

,file_path,label
0,/content/drive/MyDrive/COURSES/DS102-final-pro...,1
1,/content/drive/MyDrive/COURSES/DS102-final-pro...,1
2,/content/drive/MyDrive/COURSES/DS102-final-pro...,1
3,/content/drive/MyDrive/COURSES/DS102-final-pro...,1
4,/content/drive/MyDrive/COURSES/DS102-final-pro...,1
...,...,...
1121,/content/drive/MyDrive/COURSES/DS102-final-pro...,15
1122,/content/drive/MyDrive/COURSES/DS102-final-pro...,15
1123,/content/drive/MyDrive/COURSES/DS102-final-pro...,15
1124,/content/drive/MyDrive/COURSES/DS102-final-pro...,15


In [ ]:
df['label'].value_counts()

0     106
8     104
2      88
3      85
4      82
9      78
7      77
5      74
6      73
10     70
1      65
13     58
14     50
11     41
12     41
15     34
Name: label, dtype: int64

# Image augmentation

In [ ]:
# Number of balls to be augmented from original ones
NUM_AUGMENT = 5

In [ ]:
# for x, val in tqdm(zip(data_generator.flow_from_dataframe(df, 
#                                                     batch_size=len(df),
#                                                     x_col = 'file_path',
#                                                     y_col = 'label',
#                                                     # target_size = (24, 24),
#                                                     color_mode = 'rgb',
#                                                     save_to_dir='/content/drive/MyDrive/COURSES/DS102-final-project/ball-augmented/',     
#                                                     save_prefix='aug',
#                                                     validation_split=0.0,
#                                                     save_format='png'), range(NUM_AUGMENT - 1))):   
#     pass

Found 1126 validated image filenames belonging to 16 classes.


4it [02:10, 32.60s/it]


In [ ]:
# out_path = '/content/drive/MyDrive/COURSES/DS102-final-project/ball-augmented/'
# print(len(os.listdir(out_path)))
# # !ls -la $out_path

5630


Image augmentation's operation:
- Flip vertically
- Flip horizontally
- Rotate image
- Shear image in a range of .2
- Normalize image

In [ ]:
def augment(df):
    def augmentation(df, x_col, y_col, saved_dir, saved_prefix, num_aug):
        """
        Hàm augment bóng

        Args:
        df -- DataFrame của list bóng cần augment
        x_col -- cột chứa tên ảnh (file_path)
        y_col -- cột chứa tên nhãn (label)
        saved_dir -- folder để lưu ảnh đã được augmentated
        saved_prefix -- tiền tố của tên ảnh được lưu
        num_aug -- số lượng ảnh cần được augment từ 1 ảnh gốc

        Returns:
        res_df -- DataFrame chứa thông tin tên bóng và label sau khi đã augment
        """

        data_generator = ImageDataGenerator(
                                        vertical_flip=True,
                                        horizontal_flip=True, 
                                        rotation_range = 10,
                                        shear_range=0.2,
                                        rescale=1./255,
                                        )
        for x, val in tqdm(zip(data_generator.flow_from_dataframe(df, 
                                                        batch_size=len(df),
                                                        x_col = x_col,
                                                        y_col = y_col,
                                                        target_size=(32, 32),
                                                        color_mode = 'rgb',
                                                        save_to_dir=saved_dir,     
                                                        save_prefix=saved_prefix,
                                                        validation_split=0.0,
                                                        save_format='png'), range(num_aug))):   
            pass

    # dir_to_save = '/content/drive/MyDrive/DS102-final-project/balls-augmented-mp/'
    dir_to_save = '/content/test/'
    threshold = max(df['label'].value_counts()) - min(df['label'].value_counts()) 
    for lbl in df['label'].unique():
        # ball_label = str(lbl) if int(lbl) >= 10 else '0' + str(lbl)
        print(f'=====> Current label: {lbl}.....')
        print(f'=====> Current directory: {dir_to_save + str(lbl)}.....')
        lbl_cnt = len(df[df['label'] == lbl])
        if  lbl_cnt < threshold:
            augmentation(df[df['label']==lbl], 'file_path', 'label', dir_to_save + str(lbl) + '/', 'Augment-No.' + str(lbl) + '-', 4)
        else:
            augmentation(df[df['label']==lbl], 'file_path', 'label', dir_to_save + str(lbl) + '/', 'Augment-No.' + str(lbl) + '-', 2)

    path_augment='/content/test/'
    # path_augment = '/content/drive/MyDrive/COURSES/DS102-final-project/balls-augmented-mp/'
    df = {'file_path': [], 'label': []}
    df = pd.DataFrame(df) 
    for i in range(len(ball_type)):
        for file in os.listdir(path+ball_type[i]):
            new_row = {
                        'file_path': path_augment + ball_type[i] + '/' + file,
                        'label': ball_type[i]
                    }
            df = df.append(new_row, ignore_index=True)
    return df

In [ ]:
# # !mkdir test
# !mkdir 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 
# %cd test
# for i in range(16):
#     os.system(f'!mkdir {i}')

In [ ]:
new_df = augment(df)
new_df

=====> Current label: 1.....
=====> Current directory: /content/test/1.....
Found 65 validated image filenames belonging to 1 classes.


4it [00:01,  3.51it/s]


=====> Current label: 0.....
=====> Current directory: /content/test/0.....
Found 106 validated image filenames belonging to 1 classes.


2it [00:22, 11.32s/it]


=====> Current label: 2.....
=====> Current directory: /content/test/2.....
Found 88 validated image filenames belonging to 1 classes.


...

=====> Current label: 14.....
=====> Current directory: /content/test/14.....
Found 50 validated image filenames belonging to 1 classes.


4it [00:11,  2.81s/it]


=====> Current label: 15.....
=====> Current directory: /content/test/15.....
Found 34 validated image filenames belonging to 1 classes.


4it [00:07,  1.94s/it]


,file_path,label
0,/content/test/1/1_6.jpg,1
1,/content/test/1/1_10.jpg,1
2,/content/test/1/1_2.jpg,1
3,/content/test/1/1_4.jpg,1
4,/content/test/1/1_8.jpg,1
...,...,...
1121,/content/test/15/15_906.jpg,15
1122,/content/test/15/15_919.jpg,15
1123,/content/test/15/15_927.jpg,15
1124,/content/test/15/15_951.jpg,15


# Augment to resolve imbalanced data

In [ ]:
path = '/content/drive/MyDrive/COURSES/DS102-final-project/balls/'
ball_type = [_ for _ in os.listdir(path)]

df = {'file_path': [], 'label': []}
df = pd.DataFrame(df) 
for i in range(len(ball_type)):
    for file in os.listdir(path+ball_type[i]):
        new_row = {
                    'file_path': path + ball_type[i] + '/' + file,
                    'label': ball_type[i]
                  }
        df = df.append(new_row, ignore_index=True)
df['label'].value_counts()

0     106
8     104
2      88
3      85
4      82
9      78
7      77
5      74
6      73
10     70
1      65
13     58
14     50
12     41
11     41
15     34
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

df = train_df

In [ ]:
df.shape

(788, 2)

In [ ]:
threshold = max(df['label'].value_counts()) - min(df['label'].value_counts()) 
threshold

51

In [ ]:
dir_to_save = '/content/drive/MyDrive/DS102-final-project/balls-augmented-mp/'
for lbl in df['label'].unique():
    # ball_label = str(lbl) if int(lbl) >= 10 else '0' + str(lbl)
    print(f'=====> Current label: {lbl}.....')
    print(f'=====> Current directory: {dir_to_save + str(lbl)}.....')
    lbl_cnt = len(df[df['label'] == lbl])
    if  lbl_cnt < threshold:
        augmentation(df[df['label']==lbl], 'file_path', 'label', dir_to_save + str(lbl) + '/', 'Augment-No.' + str(lbl) + '-', 4)
    else:
        augmentation(df[df['label']==lbl], 'file_path', 'label', dir_to_save + str(lbl) + '/', 'Augment-No.' + str(lbl) + '-', 2)

In [ ]:
path = '/content/drive/MyDrive/DS102-final-project/balls-augmented-mp/'
ball_type = [_ for _ in os.listdir(path)]

df = {'file_path': [], 'label': []}
df = pd.DataFrame(df) 
for i in range(len(ball_type)):
    for file in os.listdir(path+ball_type[i]):
        new_row = {
                    'file_path': path + ball_type[i] + '/' + file,
                    'label': ball_type[i]
                  }
        df = df.append(new_row, ignore_index=True)
df['label'].value_counts()

9     245
6     245
5     240
8     228
0     228
1     215
2     201
13    195
14    190
10    174
4     168
7     165
3     153
11    150
12    140
15    125
Name: label, dtype: int64

In [ ]:
path = '/content/drive/MyDrive/COURSES/DS102-final-project/balls/'
ball_type = [_ for _ in os.listdir(path)]
ball_type.pop(0)
# for i in range(len(ball_type)):
#     for file in os.listdir(path+ball_type[i]):
#         new_row = {
#                     'file_path': path + ball_type[i] + '/' + file,
#                     'label': ball_type[i]
#                   }
#         df = df.append(new_row, ignore_index=True)



path_augment = '/content/drive/MyDrive/COURSES/DS102-final-project/balls-augmented/'
df = {'file_path': [], 'label': []}
df = pd.DataFrame(df) 
for i in range(len(ball_type)):
    for file in os.listdir(path+ball_type[i]):
        new_row = {
                    'file_path': path_augment + ball_type[i] + '/' + file,
                    'label': ball_type[i]
                  }
        df = df.append(new_row, ignore_index=True)
df

,file_path,label
0,/content/drive/MyDrive/COURSES/DS102-final-pro...,1
1,/content/drive/MyDrive/COURSES/DS102-final-pro...,1
2,/content/drive/MyDrive/COURSES/DS102-final-pro...,1
3,/content/drive/MyDrive/COURSES/DS102-final-pro...,1
4,/content/drive/MyDrive/COURSES/DS102-final-pro...,1
...,...,...
2247,/content/drive/MyDrive/COURSES/DS102-final-pro...,15
2248,/content/drive/MyDrive/COURSES/DS102-final-pro...,15
2249,/content/drive/MyDrive/COURSES/DS102-final-pro...,15
2250,/content/drive/MyDrive/COURSES/DS102-final-pro...,15
